In [1]:
# Importação das bibliotecas necessárias
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Inicialização do modelo e tokenizer
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# Função para normalização de texto
def normalize_text(text):
    import unicodedata
    import re
    if pd.isnull(text): return ""
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    text = re.sub(r'[^\w\s]', '', text).lower()
    return text

# Função para truncar texto usando o tokenizer
def truncate_text(text, max_tokens=512):
    tokens = tokenizer.encode(text, truncation=True, max_length=max_tokens, add_special_tokens=True)
    return tokenizer.decode(tokens, skip_special_tokens=True)

# Função para cálculo de embeddings
def get_embeddings(text):
    # Trunca o texto antes de passar ao modelo
    truncated_text = truncate_text(text)
    inputs = tokenizer(truncated_text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Usa apenas o primeiro token da sequência (CLS token) como representação
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

# Função para cálculo de similaridade
def calculate_similarity_matrix(embeddings_politicas, embeddings_acoes):
    # Calcula similaridades de forma vetorizada
    return cosine_similarity(embeddings_politicas, embeddings_acoes)

# Carregamento dos datasets
print("Carregando os datasets...")
politicas = pd.read_csv('Plano_Brasil_Sem_Fome.csv', encoding='utf-8')
acoes = pd.read_csv('acoes_assistencia.csv', encoding='utf-8')
print("Datasets carregados com sucesso.")

# Normalização e combinação de textos
print("Normalizando e combinando os textos das políticas e ações...")
politicas['texto_concat'] = (
    politicas['nome'] * 5 + ' ' +
    politicas['publico_alvo_nome'] * 4 + ' ' +
    politicas['publico_alvo_legislacao'] * 3 + ' ' +
    politicas['objetivos'] * 2 + ' ' +
    politicas['area'] + ' ' +
    politicas['cat_nome']
).apply(normalize_text)

acoes['texto_concat'] = (
    acoes['Título'] * 5 + ' ' +
    acoes['Descrição'] * 4 + ' ' +
    acoes['Produto'] * 3 + ' ' +
    acoes['Especificação do Produto'] * 2 + ' ' +
    acoes['Beneficiário']
).apply(normalize_text)
print("Textos normalizados e combinados.")

# Cálculo dos embeddings
print("Calculando embeddings das políticas...")
embeddings_politicas = np.stack(politicas['texto_concat'].apply(get_embeddings))
print("Embeddings das políticas calculados.")

print("Calculando embeddings das ações...")
embeddings_acoes = np.stack(acoes['texto_concat'].apply(get_embeddings))
print("Embeddings das ações calculados.")

# Processamento em lotes para cálculo de similaridades
print("Iniciando cálculo de similaridades...")
batch_size = 50  # Ajuste para sua memória disponível
results = []

for start_idx in range(0, len(politicas), batch_size):
    end_idx = min(start_idx + batch_size, len(politicas))
    print(f"Processando lote de políticas {start_idx} até {end_idx}...")

    # Extrai o lote de embeddings
    embeddings_batch = embeddings_politicas[start_idx:end_idx]

    # Calcula a similaridade do lote com todas as ações
    similarity_matrix = calculate_similarity_matrix(embeddings_batch, embeddings_acoes)

    # Adiciona os resultados
    for i, policy_idx in enumerate(range(start_idx, end_idx)):
        policy_id = politicas.iloc[policy_idx]['id']
        policy_name = politicas.iloc[policy_idx]['nome']

        for j, action_idx in enumerate(range(len(acoes))):
            action_id = acoes.iloc[action_idx]['Ação']
            action_title = acoes.iloc[action_idx]['Título']
            action_uo = acoes.iloc[action_idx]['UO']
            action_function = acoes.iloc[action_idx]['Função']
            similarity = similarity_matrix[i, j]

            results.append({
                'ID da política': policy_id,
                'Nome da política': policy_name,
                'ID da ação': action_id,
                'Título da ação': action_title,
                'UO responsável': action_uo,
                'Função relacionada': action_function,
                'Similaridade': similarity
            })

# Criação do dataframe de resultados
print("Criando dataframe de resultados...")
results_df = pd.DataFrame(results)

# Ordenação dos resultados por similaridade (decrescente)
print("Ordenando resultados por similaridade...")
results_df = results_df.sort_values(by='Similaridade', ascending=False)

# Exportação dos resultados para um arquivo CSV
print("Exportando resultados para arquivo CSV...")
results_df.to_csv('saída_correspondencias.csv', index=False, encoding='utf-8')

print("Processamento concluído. Arquivo 'saída_correspondencias.csv' gerado com sucesso.")


Carregando os datasets...
Datasets carregados com sucesso.
Normalizando e combinando os textos das políticas e ações...
Textos normalizados e combinados.
Calculando embeddings das políticas...
Embeddings das políticas calculados.
Calculando embeddings das ações...
Embeddings das ações calculados.
Iniciando cálculo de similaridades...
Processando lote de políticas 0 até 1...
Criando dataframe de resultados...
Ordenando resultados por similaridade...
Exportando resultados para arquivo CSV...
Processamento concluído. Arquivo 'saída_correspondencias.csv' gerado com sucesso.
